In [1]:
import cv2 as cv
import numpy as np
print(cv.__version__)

4.5.5


In [2]:
def show_wait_destroy(winname, img):
    cv.namedWindow(winname, cv.WINDOW_NORMAL)
    cv.imshow(winname, img)
    cv.resizeWindow(winname, 400, 300)
    cv.moveWindow(winname, 380, 200)
    cv.waitKey(0)
    cv.destroyWindow(winname)

In [3]:
image = cv.imread('sudoku_3.jpg')
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
show_wait_destroy('img', gray)

In [4]:
gray = cv.bitwise_not(gray)
bw = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, \
                                cv.THRESH_BINARY, 15, -2)
show_wait_destroy("binary", bw)

In [5]:
horizontal = np.copy(bw)
vertical = np.copy(bw)
cols = horizontal.shape[1]
horizontal_size = cols // 10
horizontalStructure = cv.getStructuringElement(cv.MORPH_RECT, (horizontal_size, 1))
# Apply morphology operations
horizontal = cv.erode(horizontal, horizontalStructure)
horizontal = cv.dilate(horizontal, horizontalStructure)
# Show extracted horizontal lines
show_wait_destroy("horizontal", horizontal)

In [6]:
rows = vertical.shape[0]
verticalsize = rows // 10
# Create structure element for extracting vertical lines through morphology operations
verticalStructure = cv.getStructuringElement(cv.MORPH_RECT, (1, verticalsize))
# Apply morphology operations
vertical = cv.erode(vertical, verticalStructure)
vertical = cv.dilate(vertical, verticalStructure)
# Show extracted vertical lines
show_wait_destroy("vertical", vertical)

In [8]:
bitwiseOr = cv.bitwise_or(horizontal,vertical)
show_wait_destroy("bitwiseOr", bitwiseOr)


In [9]:
squares = cv.bitwise_not(bitwiseOr)
show_wait_destroy("squares", squares)

In [10]:
merge = cv.bitwise_and(squares, bw)
show_wait_destroy("merge", merge)

In [11]:
import math
img = cv.imread("sudoku.png")
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
show_wait_destroy("gray", gray)
edges = cv.Canny(gray, 100,100)
show_wait_destroy("edges", edges)
lines = cv.HoughLinesP(edges, 1, math.pi/2, 2, None, 10, 1)
print(len(lines))
for line in lines[0]:
    pt1 = (line[0],line[1])
    pt2 = (line[2],line[3])
    cv.line(img, pt1, pt2, (0,0,255), 3)
show_wait_destroy("merge", img)

456


In [81]:
import cv2
image = cv2.imread('sudoku_2.png')
result = image.copy()
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
show_wait_destroy("thresh", thresh)
cols = horizontal.shape[1]
horizontal_size = cols // 5
# Detect horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size,1))
detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
show_wait_destroy("detect_horizontal", detect_horizontal)
cnts_h = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts_h = cnts_h[0] if len(cnts_h) == 2 else cnts_h[1]
# print(cnts_h)
for c in cnts_h:
    cv2.drawContours(result, [c], -1, (36,255,12), 2)
    
show_wait_destroy("result-1", result)
rows = vertical.shape[0]
verticalsize = rows // 5
# Detect vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,verticalsize))
detect_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
show_wait_destroy("detect_horizontal", detect_vertical)
cnts_v = cv2.findContours(detect_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts_v = cnts_v[0] if len(cnts_v) == 2 else cnts_v[1]
for c in cnts_v:
    cv2.drawContours(result, [c], -1, (36,255,12), 2)
    
show_wait_destroy("result-2", result)

In [16]:
show_wait_destroy("result", result)

In [111]:
print(cnts_h[6])
print(cnts_v[6])

[[[  5   2]]

 [[  5   3]]

 [[224   3]]

 [[224   2]]]
[[[  3   2]]

 [[  3 221]]

 [[  3 113]]

 [[  4 112]]

 [[  4   2]]]


In [118]:
out = cv2.rectangle(result, (37,2),(3,38), (0, 0, 255), 2)
show_wait_destroy("result", out)

In [122]:
for c in cnts_h[0]:
    print(c[0][0])

5
5
224
224


In [131]:
flat_list =[]
for list in cnts_v:
    for sublist in list:
        flat_list.append(sublist[0][0])
print(flat_list)

[221, 221, 222, 222, 186, 186, 149, 149, 113, 113, 76, 76, 39, 39, 3, 3, 3, 4, 4]


In [18]:
f1 = [sublist[0][1] for lists in cnts_h for sublist in lists]
h_max = max(f1)
h_min = min(f1)


In [19]:
print(h_max)
print(h_min)
h_splits = (h_max+h_min)//len(cnts_h)
print(h_splits)

1199
0
119


In [20]:
show_wait_destroy("result", image)

In [87]:
for points in flat_list:
    if int(points)+1 in flat_list or int(points)+2 in flat_list or points in flat_list or int(points)+3 in flat_list:
        flat_list.remove(points)
print(flat_list)

[221, 222, 186, 149, 113, 76, 39, 3, 4]


In [132]:
latest_flat=[]
for points in flat_list:
    if points not in latest_flat:
        latest_flat.append(points)
latest_flat=sorted(latest_flat)

In [133]:
print(latest_flat)

[3, 4, 39, 76, 113, 149, 186, 221, 222]


In [106]:
remove= []
for pointX in range(len(latest_flat)):
    for pointY in range(0,len(latest_flat)-1):
        if int(latest_flat[pointX])-int(latest_flat[pointY]) <=5 and int(latest_flat[pointX])-int(latest_flat[pointY]) >= 0:
            remove.append(latest_flat[pointY])
print(remove)

[221, 221, 222, 186, 149, 113, 76, 39, 3, 3]


In [120]:
remove= []
for x in latest_flat:
    y_list = [*range(x,x+10)]
    for y in y_list:
        if ( y in  latest_flat):
            remove.append(y)

In [121]:
print(remove)

[222, 186, 149, 113, 76, 39, 3, 4, 4]


In [129]:
def rem(latest_flat,current):
    latest_flat = latest_flat[:max(current - 2, 0)] + latest_flat[current:]
    print(latest_flat)
    return latest_flat

for x in latest_flat:
    rem(latest_flat,x)
#     print(x)
print(latest_flat)

[3, 76, 113, 149, 186, 221, 222]
[3, 4, 113, 149, 186, 221, 222]
[3, 4, 39, 76, 113, 149, 186, 221, 222]
[3, 4, 39, 76, 113, 149, 186, 221, 222]
[3, 4, 39, 76, 113, 149, 186, 221, 222]
[3, 4, 39, 76, 113, 149, 186, 221, 222]
[3, 4, 39, 76, 113, 149, 186, 221, 222]
[3, 4, 39, 76, 113, 149, 186, 221, 222]
[3, 4, 39, 76, 113, 149, 186, 221, 222]
[3, 4, 39, 76, 113, 149, 186, 221, 222]


In [136]:
for x  in latest_flat:
    if (int(x)+1)  in latest_flat:
        latest_flat.remove(int(x)+1)
    if (int(x)+2)  in latest_flat:
        latest_flat.remove(int(x)+2)
    if (int(x)+3)  in latest_flat:
        latest_flat.remove(int(x)+3)
print(latest_flat)

[4, 39, 76, 113, 149, 186, 221]


In [138]:
for i in range(len(latest_flat)-1):
    x = zip(latest_flat[i],latest_flat[i+1])
print(x)

TypeError: 'numpy.intc' object is not iterable

In [164]:
for x_v , y_v in zip(latest_flat, latest_flat[1:]):
    for x_h , y_h in zip(latest_flat, latest_flat[1:]):
        print(x_v,y_v,x_h,y_h)
#         x_v = 4
#         y_v = 39
        cropped_image = image[x_v:y_v ,x_h:y_h]
        gray = cv2.cvtColor(cropped_image,cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        show_wait_destroy("sample",cropped_image)
        show_wait_destroy("thresh", thresh)
        


4 39 4 39
4 39 39 76
4 39 76 113
4 39 113 149
4 39 149 186
4 39 186 221
39 76 4 39
39 76 39 76
39 76 76 113
39 76 113 149
39 76 149 186
39 76 186 221
76 113 4 39
76 113 39 76
76 113 76 113
76 113 113 149
76 113 149 186
76 113 186 221
113 149 4 39
113 149 39 76
113 149 76 113
113 149 113 149
113 149 149 186
113 149 186 221
149 186 4 39
149 186 39 76
149 186 76 113
149 186 113 149
149 186 149 186
149 186 186 221
186 221 4 39
186 221 39 76
186 221 76 113
186 221 113 149
186 221 149 186
186 221 186 221


# 